In [ ]:
# react

In [1]:
class CallbackManager(object):
    counter = 0
    observed1 = []
    observed2 = []

    @staticmethod
    def reset():
        CallbackManager.counter = 0
        CallbackManager.observed1 = []
        CallbackManager.observed2 = []

    @staticmethod
    def count(sender, value):
        CallbackManager.counter += 1

    @staticmethod
    def observe1(sender, value):
        CallbackManager.observed1.append(value)

    @staticmethod
    def observe2(sender, value):
        CallbackManager.observed2.append(value)
        

def increment(values):
    return values[0] + 1


def decrement(values):
    return values[0] - 1


def product(values):
    return values[0] * values[1]


def minimum_of_2(values):
    return values[0] + 1 if values[0] > 2 else 2

In [2]:
class Cell(object):
    def __init__(self):
        self.watchers = []
        self.value = None

    def set_value(self, value):
        if value == self.value:
            return
        self.value = value
        for w in self.watchers:
            if type(w) == tuple:
                cc, dependencies, updater_callback = w
                args = [c.value for c in dependencies]
                cc.set_value(updater_callback(args))
            else:
                w(self, self.value)

    def add_watcher(self, watcher_callback):
        if watcher_callback not in self.watchers:
            self.watchers.append(watcher_callback)

    def remove_watcher(self, watcher_callback):
        if watcher_callback in self.watchers:
            self.watchers.remove(watcher_callback)


class Reactor(object):
    def create_input_cell(self, value):
        ic = Cell()
        ic.set_value(value)
        return ic

    def create_compute_cell(self, dependencies, updater_callback):
        cc = Cell()
        for dep in dependencies:
            dep.add_watcher((cc, dependencies, updater_callback))
        args = [c.value for c in dependencies]
        cc.value = updater_callback(args)
        return cc

In [3]:
reactor = Reactor()
inputCell1 = reactor.create_input_cell(1)
computeCell1 = reactor.create_compute_cell({inputCell1}, increment)
computeCell2 = reactor.create_compute_cell({inputCell1}, decrement)
computeCell3 = reactor.create_compute_cell({computeCell1,
                                            computeCell2},
                                           product)
print(computeCell3.value)  # 0
inputCell1.set_value(3)
print(computeCell3.value)  # 8

0
8


In [4]:
reactor = Reactor()
inputCell1 = reactor.create_input_cell(1)
computeCell1 = reactor.create_compute_cell({inputCell1}, increment)
computeCell2 = reactor.create_compute_cell({inputCell1}, decrement)
computeCell3 = reactor.create_compute_cell({computeCell2}, decrement)
computeCell4 = reactor.create_compute_cell({computeCell1,
                                            computeCell3},
                                           product)
CallbackManager.reset()
computeCell4.add_watcher(CallbackManager.count)
inputCell1.set_value(3)
CallbackManager.counter  # should be 1

2

In [92]:
inputCell1.set_value(3)
CallbackManager.counter  # should be 1

2

In [ ]:
# zebra redo

- each item in data_set is assigned a unique number in range 1-5
- each condition implies equality or off-by-one (inv implies b-a==1, rather than |a-b|==1)

In [1]:
%%time
data_sets = (
    'yellow blue red ivory green',
    'norwegian ukranian englishman spaniard japanese',
    'water tea milk orange-juice coffee',
    'kool chesterfield old-gold lucky-strike parliament',
    'fox horse snails dog zebra',
)

data_specs = (
    'englishman red', 
    'spaniard dog',
    'coffee green',
    'ukranian tea',
    'ivory, green',
    'old-gold snails',
    'kool yellow',
    'milk 3',
    'norwegian 1',
    'chesterfield, fox',
    'kool, horse',
    'lucky-strike orange-juice',
    'japanese parliament',
    'norwegian, blue',
)

data_specs_inv = (
    'ivory, green',
)


def find_available_values(item):
    for ds in data_sets:
        if item in ds:
            used = [known[d] for d in ds]
            unused = [n for n in range(1, 6) if n not in used]
            break
    for ds in data_specs_inv:
        if item in ds:
            a, b = ds.split(', ')
            ka, kb = known[a], known[b]
            if item == a and kb:
                if kb - 1 in unused:
                    return [kb - 1]
                else:
                    return []
            if item == b and ka:
                if ka + 1 in unused:
                    return [ka + 1]
                else:
                    return []
    for ds in data_specs:
        if item in ds:
            if ',' in ds:
                a, b = ds.split(', ')
                if a == item:
                    a, b = b, a
                aval = known[a]
                if aval:
                    return [v for v in [aval-1, aval+1] if v in unused]
            else:
                a, b = ds.split()
                if a == item:
                    a, b = b, a
                aval = known[a]
                if aval:
                    return [aval]
    return unused


def process_specs():
    for sp in data_specs:
        clauses = sp.split(', ')
        if len(clauses) == 2:
            a, b = clauses
            if known[a]:
                a, b = b, a
            if not known[a]:
                a_poss = find_available_values(a)
                if len(a_poss) == 1:
                    known[a] = a_poss[0]
        else:
            for cl in clauses:
                eqs = cl.split()
                if len(eqs) == 2:
                    a, b = eqs
                    if b.isdigit():
                        known[a] = int(b)
                    elif known[a]:
                        known[b] == known[a]
                    elif known[b]:
                        knowns[a] == known[b]


data_sets = [ds.split() for ds in data_sets]

known = dict()
for ds in data_sets:
    for item in ds:
        known[item] = 0
                        
for _ in range(5):
    process_specs()

unknown = [k for k, v in known.items() if not v]
known_orig = known.copy()

try_item = unknown.pop(0)
try_val_lists = [[(try_item, av)] for av in find_available_values(try_item)]

found = False
while not found and try_val_lists:
    known = known_orig.copy()
    tvl = try_val_lists.pop(0)
    for item, try_val in tvl:
        known[item] = try_val   
    unknown = [k for k, v in known.items() if not v]        
    try_item = unknown.pop(0)
    for av in find_available_values(try_item):
        if unknown == []:
            found = True
        try_val_lists.append(tvl + [(try_item, av)])

known = known_orig.copy()
tvl = try_val_lists.pop()
for item, try_val in tvl:
    known[item] = try_val

def item_nationality(item):
    item_house = known[item]
    return [nat for nat in data_sets[1] if known[nat] == item_house][0]

def solution():
    return (f"It is the {item_nationality('water').capitalize()} who drinks the water.\n"
            f"The {item_nationality('zebra').capitalize()} keeps the zebra.")

print(solution())

It is the Norwegian who drinks the water.
The Japanese keeps the zebra.
CPU times: user 1.1 s, sys: 6.81 ms, total: 1.11 s
Wall time: 1.11 s


In [ ]:
# zebra, 2nd (and working) approach

In [2]:
%%time
import itertools


'''problem data setup'''


houses_pattern = [[str(n+1)]+[None]*5 for n in range(5)]

data = (
    'number 1 2 3 4 5',
    'color yellow blue red ivory green',
    'nationality norwegian ukranian englishman spaniard japanese',
    'drink water tea milk orange-juice coffee',
    'smoke kool chesterfield old-gold lucky-strike parliament',
    'pet fox horse snails dog zebra',
)

spec_texts = (
    'englishman red', 
    'spaniard dog',
    'coffee green',
    'ukranian tea',
    'ivory, green',
    'old-gold snails',
    'kool yellow',
    'milk 3',
    'norwegian 1',
    'chesterfield, fox',
    'kool, horse',
    'lucky-strike orange-juice',
    'japanese parliament',
    'norwegian, blue',
)

invariants_texts = (
    'ivory, green',
)

incomplete_indices = (3, 5)  # we are solving for a drink and a pet
complete_indices = [i for i in range(1,5) if i not in incomplete_indices]

data_dict = {}
for line in data:
    dat = line.split()
    data_dict[dat[0]] = dat[1:]

data_ordering = list(data_dict.keys())
val_key_dict = {}
for k, v in data_dict.items():
    for val in v:
        val_key_dict[val] = k


'''implementation functions'''        


def _text_to_cond(spec_text):
    cond = [None] * 6
    vals = spec_text.split()
    for val in vals:
        key = val_key_dict[val]
        cond[data_ordering.index(key)] = val
    return cond


def text_to_cond(spec_text):
    return list(map(_text_to_cond, spec_text.split(',')))


def _compare_conds(condA, condB):
    match = False
    fill = 0
    fit = True
    for cAe, cBe in zip(condA, condB):
        if cAe and cAe == cBe:
            match = True
        if bool(cAe) != bool(cBe):  # clever XOR
            fill += 1
        if cAe and cBe and cAe != cBe:
            fit = False
    if match:
        return 'match'
    elif fill == 6:
        return 'fill'
    elif fit:
        return 'fit'
    else:
        return 'fail'

    
def _invariant_is_violated(cond_list):
    for inv in invariants_lists:
        match = False
        res = [_compare_conds(inv[ci], c) for ci, c in enumerate(cond_list)]
        res_w_rev = res + [_compare_conds(inv[1-ci], c) for ci, c in enumerate(cond_list)]
        if 'match' in res_w_rev and 'fail' in res:
            return True
    return False


def _classify_cond_list_at_pos(target, cond_list, pos):
    # compound conds are left aligned
    if len(cond_list) == 2:
        if pos == 4:
            pos = 3
        if _invariant_is_violated(cond_list):
            return 'fail'
    res = [_compare_conds(target[pos+ci], c) for ci, c in enumerate(cond_list)]
    if 'fail' in res:
        return 'fail'
    if 'match' in res:
        return 'match'
    elif 'fill' in res:
        return 'fill'
    else:
        return 'fit'

    
def flip_cond_list_if_allowed(cond_list):
    if len(cond_list) == 1:
        return cond_list
    cl_rev = cond_list[::-1]
    if not _invariant_is_violated(cl_rev):
        return cl_rev
    return cond_list
        

def classify_cond_list_at_pos(target, cond_list, pos):
    if pos + len(cond_list) > len(target):
        pos -= 1
    target = target[pos:pos+len(cond_list)]
    res = _classify_cond_list_at_pos(target, cond_list, 0)
    cl_rev = flip_cond_list_if_allowed(cond_list)
    if cl_rev != cond_list:
        res_flipped = _classify_cond_list_at_pos(target, cl_rev, 0)
        if res == 'fail':
            return res_flipped
        elif res == 'fit' and res_flipped == 'fill':
            return res_flipped
        elif res == 'fill' and res_flipped == 'match':
            return res_flippped
    return res


def find_mergeable_conditions():
    # finds overlapping condition lists
    mergeable_lists = []
    for cla in all_cond_lists:
        for clb in all_cond_lists:
            if cla != clb and len(cla) < len(clb):
                for pos in (0, 1):
                    res = classify_cond_list_at_pos(clb, cla, pos)
                    if res == 'match':
                        mergeable_lists.append([clb, cla])
    return mergeable_lists


def merge_at_match(clb, cla):
    clb = clb[:]
    # a into b, if len are equal, position is 0, else must find if pos is 0 or 1
    match_pos = 0
    if len(cla) == 1 and len(clb) == 2:
        if _compare_conds(clb[1], cla[0]) == 'match':
            match_pos = 1
    for cli, cl in enumerate(cla):
        clb[match_pos + cli] = [c if c else cl[i] 
                                for i, c in enumerate(clb[match_pos + cli])]
    return clb


def merge_conditions_where_possible():
    for mcs in find_mergeable_conditions():
        clb, cla = mcs
        clb_new = (merge_at_match(clb, cla))
        all_cond_lists.remove(cla)
        clb_loc = all_cond_lists.index(clb)
        all_cond_lists[clb_loc] = clb_new   


def merge_to_houses(houses, cl, pos):
    houses = houses[:]
    houses[pos:pos+len(cl)] = merge_at_match(houses[pos:pos+len(cl)], cl)
    return houses        


def merge_matching_conds_into_houses(houses):
    houses = houses[:]
    merge_clis = []
    for hi, _ in enumerate(houses):
        for cli, cl in enumerate(all_cond_lists):
            res = classify_cond_list_at_pos(houses, all_cond_lists[cli], hi)
            if res == 'match':
                merge_clis.append((cli, hi))
    for cli, hi in sorted(merge_clis, reverse=True):
        houses = merge_to_houses(houses, all_cond_lists[cli], hi)
        del all_cond_lists[cli]
    return houses


def merge_conditions_fitting_single_spot_into_houses(houses):
    houses = houses[:]
    merge_clis = []
    for cei in complete_indices:
        clis_with_ce = [cli 
                         for cli, cl in enumerate(all_cond_lists)
                         if any([c[cei] for c in cl])]
        for hi in range(5):
            if not houses[hi][cei]:  # not already populated
                fit_res = [classify_cond_list_at_pos(houses, all_cond_lists[cli], hi) 
                           for cli in clis_with_ce]
                if fit_res.count('fit') == 1:
                    fit_cli = clis_with_ce[fit_res.index('fit')]
                    merge_clis.append((fit_cli, hi))
    for cli, hi in sorted(merge_clis, reverse=True):
        houses = merge_to_houses(houses, all_cond_lists[cli], hi)
        del all_cond_lists[cli]
    return houses


def see_houses_and_all_cond_list():   
    for hi, h in enumerate(houses):
        print('-' * 71, hi)
        print('|'.join([f'{ce:<12}' if ce else ' '*12 for ce in h]))
    print('\n\n')
    for cli, cl in enumerate(all_cond_lists):
        print('-' * 71, cli)
        for c in cl:
            print('|'.join([f'{ce:<12}' if ce else ' '*12 for ce in c]))

            
def test_trial_position(tps):
    global houses_solution
    test_houses = houses[:]
    try:
        for cli, clp in enumerate(tps):
            test_houses = merge_to_houses(test_houses, all_cond_lists[cli], clp)
        none_count = sum([h.count(None) for h in test_houses])
        if none_count == 2:
            houses_solution = test_houses
        return none_count
    except IndexError:
        pass

    
def solution():
    global houses_solution
    drink_index = 3
    pet_index = 5
    for house in houses_solution:
        if house[drink_index] is None:
            water_drinker = house[2].capitalize()
        if house[pet_index] is None:
            zebra_keeper = house[2].capitalize()
    return (f'It is the {water_drinker} who drinks the water.\n'
            f'The {zebra_keeper} keeps the zebra.')


'''execution'''


all_cond_lists = list(map(text_to_cond, spec_texts))
invariants_lists = list(map(text_to_cond, invariants_texts))
merge_conditions_where_possible()
houses = houses_pattern[:]
houses = merge_matching_conds_into_houses(houses)
houses = merge_conditions_fitting_single_spot_into_houses(houses)

houses_solution = []

fit_locs = [[hi for hi in range(5) if classify_cond_list_at_pos(houses, cl, hi) == 'fit']
            for cl in all_cond_lists]
trial_positions = list(itertools.product(*fit_locs))
trial_positions = [tps for tps in trial_positions
                   if set(tps) == set((0, 1, 2, 3, 4)) ]
for tp in trial_positions:
    if test_trial_position(tp) == 2:
        break

all_cond_lists = [flip_cond_list_if_allowed(cond_list) for cond_list in all_cond_lists]
fit_locs = [[hi for hi in range(5) if classify_cond_list_at_pos(houses, cl, hi) == 'fit']
            for cl in all_cond_lists]
trial_positions = list(itertools.product(*fit_locs))
trial_positions = [tps for tps in trial_positions
                   if set(tps) == set((0, 1, 2, 3, 4)) ]
for tp in trial_positions:
    if test_trial_position(tp) == 2:
        break
        
houses = houses_solution
#see_houses_and_all_cond_list()
print(solution())

It is the Norwegian who drinks the water.
The Japanese keeps the zebra.
CPU times: user 55.7 ms, sys: 96 µs, total: 55.8 ms
Wall time: 55.9 ms
